In [1]:
import sys
sys.path.append("..")
import os
os.environ["MKL_NUM_THREADS"] = "3" # "6"
os.environ["OMP_NUM_THREADS"] = "2" # "4"
os.environ["NUMEXPR_NUM_THREADS"] = "3" # "6"
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
import glob

In [2]:
source_folder = r"../input/plant-pathology-2021-fgvc8"

test = glob.glob(os.path.join(source_folder, "test_images","*"))

print(test)
if len(test)>3: 
    GET_CV = False
    
else: 
    GET_CV =True
    print('this submission notebook will compute CV score, but commit notebook will not')


['../input/plant-pathology-2021-fgvc8/test_images/ad8770db05586b59.jpg', '../input/plant-pathology-2021-fgvc8/test_images/c7b03e718489f3ca.jpg', '../input/plant-pathology-2021-fgvc8/test_images/85f8cb619c66b863.jpg']
this submission notebook will compute CV score, but commit notebook will not


In [3]:
import sys

sys.path.append('../input/classification-nn-pytorch')

In [4]:
#Preliminaries
from tqdm import tqdm
import math
import random
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

import cv2
cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

# albumentations for augs
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# torch
import torch
from classification_nn_pytorch import EfficientNet
import torch.nn as nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
import ttach as tta


# Config

In [5]:
class CFG:
    root = r"../input/plant-pathology-2021-fgvc8"

    dup_csv_path = r"../input/pp-trained-models/duplicates.csv"
    classes = [
        'complex', 
        'frog_eye_leaf_spot', 
        'powdery_mildew', 
        'rust', 
        'scab',
        'healthy'
    ]

    pretrained_path = None

    models_path = r"../input/pp-trained-models"

    model_name = 'efficientnet-b4' 

    lr_rate = 1e-4
    seed = 42
    num_gpus = 1
    num_workers = min(8,num_gpus * 2) if num_gpus > 1 else 2
    batch_size = 16 * num_gpus
    img_size = 256
    folds = 5
    transform = True
    epochs = 100
    patient = 10

    tta_option = tta.Compose([
                    tta.Rotate90([0, 90, 180, 270])
                ])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Preprocess

In [6]:
class Preprocess:
    def __init__(self):
        pass

    def get_train_transforms():
        return A.Compose(
            [
                A.OneOf([
                    A.RGBShift(),
                    A.ChannelShuffle(),
                ], p=0.2),

                A.RandomResizedCrop(CFG.img_size, CFG.img_size, scale=(0.9, 1), p=1),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.ShiftScaleRotate(p=0.5),
                A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=50, p=0.7),
                A.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),
                A.CLAHE(clip_limit=(1,4), p=0.5),

                A.OneOf([
                    A.OpticalDistortion(distort_limit=1.0),
                    A.GridDistortion(num_steps=5, distort_limit=0.7),
                    A.ElasticTransform(alpha=3),
                ], p=0.2),

                A.OneOf([
                    A.GaussNoise(var_limit=[10, 50]),
                    A.GaussianBlur(),
                    A.MotionBlur(),
                    A.MedianBlur(),
                ], p=0.2),

                A.OneOf([
                    # A.RandomSunFlare(), 
                    A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.5, alpha_coef=0.1),
                    A.RandomBrightness(limit=0.3, p=1),
                ], p=0.2),

                A.Resize(CFG.img_size, CFG.img_size, always_apply=True),

                A.OneOf([
                    A.JpegCompression(),
                    A.Downscale(scale_min=0.1, scale_max=0.15),
                ], p=0.2),

                A.IAAPiecewiseAffine(p=0.2),
                A.IAASharpen(p=0.2),
                A.Cutout(max_h_size=int(CFG.img_size * 0.1), max_w_size=int(CFG.img_size * 0.1), num_holes=5, p=0.5),
                A.Normalize(),
                ToTensorV2(p=1.0)
            ]
        )

    def get_valid_transforms():
        
        return A.Compose(
        [
            A.Resize(CFG.img_size, CFG.img_size, always_apply=True),
            A.Normalize(),
            ToTensorV2(p=1.0)
        ])

# Dataset

In [7]:
class PPDataset(Dataset):
    def __init__(self, csv, transforms=None, training=True):
        self.csv = csv.reset_index()
        self.augmentations = transforms
        self.training = training

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, index):
        row = self.csv.iloc[index]
        
        image = cv2.imread(row.filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']
            
        if self.training:
            return image, torch.tensor(row[1:len(CFG.classes)+1])
        
        else:
            return image, torch.tensor(1)

# Create model

In [8]:
class Create_model:
    def __init__(self, CFG, pretrain_option=True):
        self.config = CFG
        self.pretrain_option = pretrain_option

    def _build_model(self):

        if self.pretrain_option:
            print("Loading pretrained :", self.config.pretrained_path)
            model = EfficientNet.from_pretrained(
                                                    self.config.model_name,
                                                    weights_path=self.config.pretrained_path,
                                                    advprop=False,
                                                    num_classes=len(self.config.classes),
                                                    image_size=self.config.img_size
                                                )
        
        else:
            print(f'Buidling Model Backbone for {self.config.model_name} model.')
            model =  EfficientNet.from_name(
                                                self.config.model_name,
                                                num_classes=len(self.config.classes),
                                                image_size=self.config.img_size
                                            )

        return model


In [9]:
class Main_session:
    def __init__(self, GET_CV, CFG):
        self.GET_CV = GET_CV
        self.config = CFG
        
    def read_dataset(self):
        if self.GET_CV:
            df = pd.read_csv(os.path.join(self.config.root, 'train.csv'), index_col='image')
            init_len = len(df)
            
            with open(self.config.dup_csv_path, 'r') as file:
                duplicates = [x.strip().split(",") for x in file.readlines()]

            for row in duplicates:
                unique_labels = df.loc[row].drop_duplicates().values
                if len(unique_labels)  == 1:
                    df = df.drop(row[1:], axis=0)

                else:
                    df = df.drop(row, axis=0)

            print(f"Dropping {init_len - len(df)} duplicates samples")

            original_labels = df['labels'].values.copy()
            
            df['labels'] = [x.split(' ') for x in df['labels']]

            labels = MultiLabelBinarizer(classes=self.config.classes).fit_transform(df['labels'].values)

            df = pd.DataFrame(columns=self.config.classes, data=labels, index=df.index)
            
            skf = StratifiedKFold(n_splits=self.config.folds, shuffle=True, random_state=self.config.seed)
            # skf.get_n_splits(df.index, df['label_group'])
            np_fold = np.zeros((len(df),))
            df['fold'] = 0

            for i, (train_index, test_index) in enumerate(skf.split(df.index, original_labels)):

                df['fold'].iloc[test_index] = i
                np_fold[test_index] = i

            # df['filepath'] = df.index.apply(lambda x: os.path.join(CFG.root, 'train_images',x))
            value_counts = lambda x: pd.Series.value_counts(x, normalize=True)
            
            df_check = df.drop(columns=['fold'])
            # print(df_check[np_fold == 0].apply(value_counts).loc[1])

            df_occurence = pd.DataFrame({
                "origin": df_check.apply(value_counts).loc[1],
                'fold_0': df_check[np_fold == 0].apply(value_counts).loc[1],
                'fold_1': df_check[np_fold == 1].apply(value_counts).loc[1],
                'fold_2': df_check[np_fold == 2].apply(value_counts).loc[1],
                'fold_3': df_check[np_fold == 3].apply(value_counts).loc[1],
                'fold_4': df_check[np_fold == 4].apply(value_counts).loc[1],
            })

            # print(df_occurence)
            df['image'] = df.index
            df['origin_labels'] = original_labels
            df['filepath'] = df['image'].apply(lambda x: os.path.join(self.config.root, 'train_images',x))

        else:
            test_path = glob.glob(os.path.join(source_folder, "test_images","*"))
            test_images = [path.split("/")[-1] for path in test_path]
            df = pd.DataFrame(columns=['image'], data=test_images)

            df['filepath'] = df['image'].apply(lambda x: os.path.join(self.config.root, 'test_images',x))

        return df

    def get_predict_results(self):
        data = self.read_dataset()
        data = data.reset_index(drop=True)
        
        list_models = [0] * self.config.folds
        
        probs_dict = {}
        
        for i in [0]:
            probs_list = []
            preds_list = []
            
            model = Create_model(self.config, pretrain_option=False)
            
            list_models[i] = model._build_model()
            model_path = os.path.join(self.config.models_path, f'model_{self.config.model_name}_IMG_SIZE_{self.config.img_size}_f{i}.pth')
            
            list_models[i].load_state_dict(torch.load(model_path), strict=False)
            list_models[i].to(device=self.config.device)
            list_models[i].eval()
            
            if self.config.num_gpus > 1:
                list_models[i] = tta.ClassificationTTAWrapper(list_models[i].module, self.config.tta_option)
                list_models[i] = nn.DataParallel(list_models[i])

            else:
                list_models[i] = tta.ClassificationTTAWrapper(list_models[i], self.config.tta_option)
            
            list_models[i].eval()
            
            image_dataset = PPDataset(data, transforms=Preprocess.get_valid_transforms(), training= self.GET_CV)

            image_loader = DataLoader(
                image_dataset,
                batch_size=self.config.batch_size,
                pin_memory=True,
                drop_last=False,
                num_workers=self.config.num_workers
            )

            with torch.no_grad():
                for img, label in tqdm(image_loader):
                    img = img.to(self.config.device)
                    label = label.to(self.config.device)
                    outputs = list_models[i](img)

                    propability = torch.nn.Softmax(dim = 1)(outputs)
                    propability = propability.detach().cpu().numpy()
                    probs_list.append(propability)
                    preds_list.extend([np.array(self.config.classes)[np.where(prob > 1 / len(self.config.classes))] for prob in propability])
                    
            del model
            probs_list = np.concatenate(probs_list)
            preds_list = np.array(preds_list)
            probs_dict[f"fold_{i}"] = probs_list
            data[f"fold_{i}"] = preds_list
            import gc
            gc.collect()
            
        return probs_dict, data


In [10]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_true = y_true.apply(lambda x: len(x)).values
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_true + len_y_pred)
    
    return f1

In [11]:
def combine_predictions(row):
    temp_fold = [0]
    x = np.concatenate([row[f'fold_{i}'] for i in range(len(temp_fold))])

    # values, counts = np.unique(x, return_counts=True)
    
    # x = [values[i] for i in range(len(values)) if counts[i] >= 3]
    
    return ' '.join(x)

In [12]:
session = Main_session(GET_CV, CFG)
_, pdata = session.get_predict_results()

Dropping 77 duplicates samples
Buidling Model Backbone for efficientnet-b4 model.


100%|██████████| 1160/1160 [36:56<00:00,  1.91s/it]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [13]:
if GET_CV:
    pdata['labels'] = pdata.apply(combine_predictions, axis = 1)
    pdata['f1'] = f1_score(pdata['origin_labels'], pdata['labels'])
    score = pdata['f1'].mean()
    print(f"Our final f1 cv score is {score}")
    pdata[['image', 'labels']].to_csv('submission.csv', index = False)

else:

    pdata['labels'] = pdata.apply(combine_predictions, axis = 1)
    pdata[['image', 'labels']].to_csv('submission.csv', index = False)

Our final f1 cv score is 0.9431959040689841


In [14]:
pdata[['image', 'labels']]

,image,labels
0,800113bb65efe69e.jpg,complex powdery_mildew healthy
1,8002cb321f8bfcdf.jpg,complex frog_eye_leaf_spot scab
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex
...,...,...
18550,fffb900a92289a33.jpg,healthy
18551,fffc488fa4c0e80c.jpg,scab
18552,fffc94e092a59086.jpg,rust
18553,fffe105cf6808292.jpg,scab
